## CONV-LSTMAE

In [2]:
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, timedelta
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input, LSTM,Dense,RepeatVector, TimeDistributed , LeakyReLU
from sklearn.metrics import f1_score, make_scorer, mean_squared_error
import tensorflow as tf
import pandas as pd
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input,Conv2D, Conv1D, Conv1DTranspose,MaxPooling1D
from tensorflow.keras.layers import  AveragePooling1D, UpSampling1D
import os
import numpy as np
from tensorflow.keras.layers import ConvLSTM1D, Conv1D, BatchNormalization, RepeatVector, TimeDistributed, Reshape,Attention, GlobalMaxPooling1D
import random
import pickle
import statistics
from tcn import TCN
import ast
def Conv_LstmAEModel(timesteps = 1, n_features=137, layer1=100, layer2=60, encoding_dim=4):
    input = Input(shape=(timesteps, n_features))
    x = Conv1D(filters=layer1, activation="relu",kernel_size=3, padding="same")(input)
    x = LSTM(layer2,  activation="relu", return_sequences=True)(x)
    x = LSTM(encoding_dim, activation="relu", return_sequences=True)(x)
    x = LSTM(layer2, activation="relu", return_sequences=True)(x)
    x = Conv1DTranspose(filters=layer1,  activation="relu",kernel_size=3, padding="same")(x)
    x = Conv1DTranspose(filters=n_features,  activation="sigmoid",kernel_size=3, padding="same")(x)
    autoencoder = Model(input, x)
    autoencoder.compile(optimizer="adam", loss="mse")
    autoencoder.summary()
    return autoencoder
def Conv_LstmAttnAEModel(timesteps = 1, n_features=137, layer1=100, layer2=60, encoding_dim=4):
    input = Input(shape=(timesteps, n_features))
    x = Conv1D(filters=layer1, activation="relu",kernel_size=3, padding="same")(input)
    x = LSTM(layer2,  activation="relu", return_sequences=True)(x)
    x = LSTM(encoding_dim, activation="relu", return_sequences=True)(x)
    attention = Attention()([x, x])
    attention = GlobalMaxPooling1D()(attention)
    x = Reshape((1, encoding_dim))(attention)
    x = LSTM(layer2, activation="relu", return_sequences=True)(x)
    x = Conv1DTranspose(filters=layer1,  activation="relu",kernel_size=3, padding="same")(x)
    x = Conv1DTranspose(filters=n_features,  activation="sigmoid",kernel_size=3, padding="same")(x)
    autoencoder = Model(input, x)
    autoencoder.compile(optimizer="adam", loss="mse")
#     autoencoder.summary()
    return autoencoder
def temporalize(X, lookback=8):

    output_X = []
    for i in range(len(X) - lookback - 1):
        t = []
        for j in range(1, lookback + 1):
            t.append(X[[(i + j + 1)], :])
        output_X.append(t)
    return np.squeeze(np.array(output_X))
def getHyperParams(file, df_hyp):
    param_dict = ast.literal_eval(df_hyp.loc[df_hyp['Inverter'] == file]['Feature Importance'].values[0])
    return param_dict['model__batch_size'], param_dict['model__encoding_dim'], param_dict['model__epochs'],param_dict['model__layer1'], param_dict['model__layer2'] 

def getPIDData():
    path_errors = '/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/errortimestamps.csv'
    path_features = '/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/featureslist.csv'
    df_errors = pd.read_csv(path_errors)
    df_features = pd.read_csv(path_features)
    total_features = list(df_features[(df_features['True/False'] == 'TRUE') | (df_features['True/False'] == 'NotAvailable')]['Features'])
    tot_feat = pd.read_csv('/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/totfeat.csv')
    tot_feat = list(tot_feat['feat'])
    xtrain_dfs = {}
    xtest_dfs = {}
    ytrain_dfs = {}
    ytest_dfs = {}
    for inverter in range(19):
        inv = 'inv_'+str(inverter)
        tot_feat.append('ErrBits')
        df_inv_0 = pd.read_csv('/mnt/work/digitwin/subset_data_for_experiment/downsampledata_PandasMeanNew/'+str(inverter)+'.csv')
        df_inv_0['Timestamp'] = pd.to_datetime(df_inv_0['Timestamp'])
        df_inv_0.sort_values(by='Timestamp', inplace=True)
        df_inv_0 = df_inv_0.set_index(df_inv_0['Timestamp'])
        df_inv_0.shape


        format = '%Y-%m-%d %H:%M:%S'
        error_date = datetime.strptime(df_errors[(df_errors['Inverter'] == inverter) & (df_errors['had_failure'] == True)]['failure_time'].values[0][0:19], format)
        start_date = error_date-timedelta(days= 44)
        end_date = error_date+timedelta(days= 1*30)
        split_date = error_date-timedelta(days= 1*30)

        df_inv_0 = df_inv_0[(df_inv_0.index > start_date) & (df_inv_0.index < end_date)]
        df_inv_0 = pd.get_dummies(df_inv_0)

        for feat in tot_feat:
            if feat not in df_inv_0.columns:
                df_inv_0[feat] = 0
        df_inv_0 = df_inv_0[tot_feat]
        df_inv_0 = df_inv_0.dropna()
        df_target = df_inv_0[['ErrBits']]
        tot_feat.remove('ErrBits')
        df_inv_0 = df_inv_0[tot_feat]

        xtrain, xtest = df_inv_0[(df_inv_0.index >= start_date) & (df_inv_0.index< split_date)], df_inv_0[(df_inv_0.index >= split_date) & (df_inv_0.index< end_date)]
        ytrain, ytest = df_target[(df_target.index >= start_date) & (df_target.index< split_date)], df_target[(df_target.index >= split_date) & (df_target.index< end_date)]
        xtrain_dfs[inv] = xtrain
        xtest_dfs[inv] = xtest
        ytrain_dfs[inv] = ytrain
        ytest_dfs[inv] = ytest
    return xtrain_dfs, xtest_dfs, ytrain_dfs, ytest_dfs
def get_train_test_data(inv, xtrain_dfs, xtest_dfs, ytrain_dfs, ytest_dfs):
    xtrain, xtest = xtrain_dfs[inv], xtest_dfs[inv]
    ytrain, ytest = ytrain_dfs[inv], ytest_dfs[inv]
    X_scaler = MinMaxScaler()
    X_scaler.fit(xtrain)
    xtrain = X_scaler.transform(xtrain)
    xtest = X_scaler.transform(xtest)
    X_scalar = MinMaxScaler()
    X_scalar.fit(xtrain)
    xtrain, xtest = X_scalar.transform(xtrain), X_scalar.transform(xtest)
    return xtrain, xtest, ytrain, ytest

def other_evaluation(xtrain, xtest, ytest, ytrain, model):
    test_enc = model.predict(xtest)
    train_enc = model.predict(xtrain)
    test_mses = np.mean(np.power(flatten(xtest)- flatten(test_enc), 2), axis=1)
    train_mses = np.mean(np.power(flatten(xtrain)- flatten(train_enc), 2), axis=1)
    test_maes = np.mean(flatten(xtest)- flatten(test_enc), axis=1)
    train_maes = np.mean(flatten(xtrain)- flatten(train_enc), axis=1)
    mse_threshold = np.max(train_mses)
    mae_threshold = np.max(train_maes)
    ypred_mse = np.where(test_mses>mse_threshold,1,0)
    ypred_mae = np.where(test_maes>mae_threshold,1,0)
    test_df = pd.DataFrame(ytest)
    test_df['mse_scores'] = test_mses
    test_df['ytrue'] = np.where(test_df['ErrBits']>0, 1, 0)
    test_df['mae_scores'] = test_maes
    test_df['ypred_mse'] = ypred_mse
    test_df['ypred_mae'] = ypred_mae
    test_df = test_df[['ytrue','ypred_mse','ypred_mae','mse_scores', 'mae_scores']]
    
    train_df = pd.DataFrame(ytrain)
    train_df['mse_scores'] = train_mses
    train_df['ytrue'] = np.where(train_df['ErrBits']>0, 1, 0)
    train_df['mae_scores'] = train_maes
    train_df = train_df[['ytrue','mse_scores', 'mae_scores']]
    return test_df, train_df
def data_unimodel(source_list, xtrain_dfs):
    train_merge = pd.DataFrame()
    for inverter in source_list:
        X = xtrain_dfs[f'inv_{inverter}']
        X_scaler = MinMaxScaler()
        X_scaler.fit(X)
        xtrain = X_scaler.transform(X)
        df_train = pd.DataFrame(xtrain)
        train_merge = pd.concat([train_merge, df_train], axis=0)
    train_merge_df = np.array(train_merge)
    return train_merge_df
def flatten(X):
    flattened_X = np.empty((X.shape[0], X.shape[2])) 
    for i in range(X.shape[0]):
        flattened_X[i] = X[i, (X.shape[1]-1), :]
    return(flattened_X)
source_list = [12, 4, 6, 14, 10, 18, 5, 8, 3, 0, 11, 2]
total_list = [0, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 17, 18]

xtrain_dfs, xtest_dfs, ytrain_dfs, ytest_dfs = getPIDData()
for i in range(1,4):
    df_hyp = pd.read_csv('/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/Conv_LstmAE_HYP.csv')
    PID_RESULTSPATH_TOTAL_TRAIN = f'/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/Results/Iter{i}/train/'
    PID_RESULTSPATH_TOTAL_TEST = f'/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/Results/Iter{i}/test/'
    callback1 = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, verbose=1)
    lookback = 8
    source_dict_results_test =  {}
    source_dict_results_train = {}
    for file in total_list:
        inv = f'inv_{file}'
        xtrain, xtest, ytrain, ytest = get_train_test_data(inv, xtrain_dfs, xtest_dfs, ytrain_dfs, ytest_dfs)
        xtrain = xtrain.reshape(xtrain.shape[0], 1, xtrain.shape[1])
        xtest = xtest.reshape(xtest.shape[0], 1, xtest.shape[1])
#         ytrain, ytest = ytrain[lookback+1:], ytest[lookback+1:]
        model__batch_size,model__encoding_dim, model__epochs, model__layer1, model__layer2 = getHyperParams(file, df_hyp)                                               
        model  = Conv_LstmAEModel(layer1=model__layer1, layer2=model__layer2, encoding_dim=model__encoding_dim)
        model.fit(xtrain, xtrain, epochs=model__epochs, batch_size=model__batch_size, verbose=2,validation_split=0.1, callbacks=callback1)
        test_df, train_df = other_evaluation(xtrain, xtest, ytest, ytrain, model)
        source_dict_results_test[file] = test_df
        source_dict_results_train[file] = train_df
    with open(PID_RESULTSPATH_TOTAL_TEST+'test_STL_Conv_LstmAE.pkl', 'wb') as f:
        pickle.dump(source_dict_results_test, f)
    with open(PID_RESULTSPATH_TOTAL_TRAIN+'train_STL_Conv_LstmAE.pkl', 'wb') as f:
        pickle.dump(source_dict_results_train, f)
    train_merge_df = data_unimodel(source_list, xtrain_dfs)
    random.shuffle(train_merge_df)
    train_merge_df = train_merge_df.reshape(train_merge_df.shape[0], 1, train_merge_df.shape[1])

    model = Conv_LstmAEModel(layer1=64, layer2=32, encoding_dim=32)
    model.fit(train_merge_df, train_merge_df, epochs=200, batch_size=128, verbose=2,validation_split=0.1, callbacks=callback1)
    lookback = 8
    mtl_dict_results_test =  {}
    mtl_dict_results_train = {}
    for file in total_list:
        inv = 'inv_'+str(file)
        xtrain, xtest, ytrain, ytest = get_train_test_data(inv, xtrain_dfs, xtest_dfs, ytrain_dfs, ytest_dfs)
        xtrain = xtrain.reshape(xtrain.shape[0], 1, xtrain.shape[1])
        xtest = xtest.reshape(xtest.shape[0], 1, xtest.shape[1])
#         ytrain, ytest = ytrain[lookback+1:], ytest[lookback+1:]
        test_df, train_df = other_evaluation(xtrain, xtest, ytest, ytrain, model)
        mtl_dict_results_test[file] = test_df
        mtl_dict_results_train[file] = train_df
    with open(PID_RESULTSPATH_TOTAL_TEST+'test_MTL_Conv_LstmAE.pkl', 'wb') as f:
        pickle.dump(mtl_dict_results_test, f)
    with open(PID_RESULTSPATH_TOTAL_TRAIN+'train_MTL_Conv_LstmAE.pkl', 'wb') as f:
        pickle.dump(mtl_dict_results_train, f)

        

2023-09-14 10:29:52.143382: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-09-14 10:29:52.143758: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-09-14 10:29:52.143855: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-09-14 10:29:52.143945: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2023-09-14 10:29:52.176693: W tensorflow/stream_executor/platform/default/dso_loader.cc:64

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1, 137)]          0         
                                                                 
 conv1d (Conv1D)             (None, 1, 64)             26368     
                                                                 
 lstm (LSTM)                 (None, 1, 32)             12416     
                                                                 
 lstm_1 (LSTM)               (None, 1, 32)             8320      
                                                                 
 lstm_2 (LSTM)               (None, 1, 32)             8320      
                                                                 
 conv1d_transpose (Conv1DTra  (None, 1, 64)            6208      
 nspose)                                                         
                                                             

Epoch 82/200
230/230 - 1s - loss: 5.1596e-04 - val_loss: 0.0014 - 1s/epoch - 5ms/step
Epoch 83/200
230/230 - 1s - loss: 5.2183e-04 - val_loss: 0.0012 - 1s/epoch - 5ms/step
Epoch 84/200
230/230 - 1s - loss: 5.0970e-04 - val_loss: 0.0011 - 1s/epoch - 5ms/step
Epoch 85/200
230/230 - 1s - loss: 5.0054e-04 - val_loss: 0.0012 - 1s/epoch - 5ms/step
Epoch 86/200
230/230 - 1s - loss: 5.0474e-04 - val_loss: 0.0012 - 1s/epoch - 5ms/step
Epoch 87/200
230/230 - 1s - loss: 4.9852e-04 - val_loss: 0.0015 - 1s/epoch - 5ms/step
Epoch 88/200
230/230 - 1s - loss: 4.9641e-04 - val_loss: 0.0011 - 1s/epoch - 5ms/step
Epoch 89/200
230/230 - 1s - loss: 4.8279e-04 - val_loss: 0.0013 - 1s/epoch - 5ms/step
Epoch 90/200
230/230 - 1s - loss: 4.8798e-04 - val_loss: 0.0012 - 1s/epoch - 5ms/step
Epoch 91/200
230/230 - 1s - loss: 4.8494e-04 - val_loss: 0.0011 - 1s/epoch - 5ms/step
Epoch 92/200
230/230 - 1s - loss: 4.7466e-04 - val_loss: 0.0011 - 1s/epoch - 5ms/step
Epoch 93/200
230/230 - 1s - loss: 4.6703e-04 - val_los

Epoch 176/200
230/230 - 1s - loss: 3.3340e-04 - val_loss: 9.6877e-04 - 1s/epoch - 5ms/step
Epoch 177/200
230/230 - 1s - loss: 3.1201e-04 - val_loss: 8.7819e-04 - 1s/epoch - 5ms/step
Epoch 178/200
230/230 - 1s - loss: 3.1806e-04 - val_loss: 9.4557e-04 - 1s/epoch - 5ms/step
Epoch 179/200
230/230 - 1s - loss: 3.2760e-04 - val_loss: 0.0010 - 1s/epoch - 5ms/step
Epoch 180/200
230/230 - 1s - loss: 3.1037e-04 - val_loss: 9.7502e-04 - 1s/epoch - 5ms/step
Epoch 181/200
230/230 - 1s - loss: 3.1382e-04 - val_loss: 9.4815e-04 - 1s/epoch - 5ms/step
Epoch 182/200
230/230 - 1s - loss: 3.1260e-04 - val_loss: 9.7226e-04 - 1s/epoch - 5ms/step
Epoch 183/200
230/230 - 1s - loss: 3.1973e-04 - val_loss: 9.6642e-04 - 1s/epoch - 5ms/step
Epoch 184/200
230/230 - 1s - loss: 3.0675e-04 - val_loss: 9.2548e-04 - 1s/epoch - 5ms/step
Epoch 185/200
230/230 - 1s - loss: 3.0206e-04 - val_loss: 0.0011 - 1s/epoch - 5ms/step
Epoch 186/200
230/230 - 1s - loss: 3.0897e-04 - val_loss: 9.3985e-04 - 1s/epoch - 5ms/step
Epoch 1

Epoch 56/200
230/230 - 1s - loss: 9.5443e-04 - val_loss: 0.0018 - 1s/epoch - 5ms/step
Epoch 57/200
230/230 - 1s - loss: 8.6000e-04 - val_loss: 0.0018 - 1s/epoch - 5ms/step
Epoch 58/200
230/230 - 1s - loss: 8.5800e-04 - val_loss: 0.0017 - 1s/epoch - 5ms/step
Epoch 59/200
230/230 - 1s - loss: 8.3912e-04 - val_loss: 0.0017 - 1s/epoch - 5ms/step
Epoch 60/200
230/230 - 1s - loss: 8.3063e-04 - val_loss: 0.0018 - 1s/epoch - 5ms/step
Epoch 61/200
230/230 - 1s - loss: 8.3301e-04 - val_loss: 0.0018 - 1s/epoch - 5ms/step
Epoch 62/200
230/230 - 1s - loss: 8.3191e-04 - val_loss: 0.0017 - 1s/epoch - 5ms/step
Epoch 63/200
230/230 - 1s - loss: 8.2164e-04 - val_loss: 0.0016 - 1s/epoch - 5ms/step
Epoch 64/200
230/230 - 1s - loss: 7.9839e-04 - val_loss: 0.0016 - 1s/epoch - 5ms/step
Epoch 65/200
230/230 - 1s - loss: 7.9315e-04 - val_loss: 0.0016 - 1s/epoch - 5ms/step
Epoch 66/200
230/230 - 1s - loss: 7.7936e-04 - val_loss: 0.0016 - 1s/epoch - 5ms/step
Epoch 67/200
230/230 - 1s - loss: 7.7657e-04 - val_los

Epoch 151/200
230/230 - 1s - loss: 3.6679e-04 - val_loss: 0.0010 - 1s/epoch - 5ms/step
Epoch 152/200
230/230 - 1s - loss: 3.6645e-04 - val_loss: 9.7932e-04 - 1s/epoch - 5ms/step
Epoch 153/200
230/230 - 1s - loss: 3.6712e-04 - val_loss: 0.0010 - 1s/epoch - 5ms/step
Epoch 154/200
230/230 - 1s - loss: 3.6136e-04 - val_loss: 0.0011 - 1s/epoch - 5ms/step
Epoch 155/200
230/230 - 1s - loss: 3.5986e-04 - val_loss: 0.0010 - 1s/epoch - 5ms/step
Epoch 156/200
230/230 - 1s - loss: 3.6072e-04 - val_loss: 9.9316e-04 - 1s/epoch - 5ms/step
Epoch 157/200
230/230 - 1s - loss: 3.5189e-04 - val_loss: 0.0011 - 1s/epoch - 5ms/step
Epoch 158/200
230/230 - 1s - loss: 3.6461e-04 - val_loss: 0.0011 - 1s/epoch - 5ms/step
Epoch 159/200
230/230 - 1s - loss: 3.5732e-04 - val_loss: 9.6903e-04 - 1s/epoch - 5ms/step
Epoch 160/200
230/230 - 1s - loss: 3.8599e-04 - val_loss: 9.8740e-04 - 1s/epoch - 5ms/step
Epoch 161/200
230/230 - 1s - loss: 3.4384e-04 - val_loss: 0.0010 - 1s/epoch - 5ms/step
Epoch 162/200
230/230 - 1s 

Epoch 29/200
230/230 - 1s - loss: 0.0018 - val_loss: 0.0023 - 1s/epoch - 5ms/step
Epoch 30/200
230/230 - 1s - loss: 0.0016 - val_loss: 0.0023 - 1s/epoch - 5ms/step
Epoch 31/200
230/230 - 1s - loss: 0.0016 - val_loss: 0.0022 - 1s/epoch - 5ms/step
Epoch 32/200
230/230 - 1s - loss: 0.0015 - val_loss: 0.0021 - 1s/epoch - 5ms/step
Epoch 33/200
230/230 - 1s - loss: 0.0015 - val_loss: 0.0020 - 1s/epoch - 5ms/step
Epoch 34/200
230/230 - 1s - loss: 0.0015 - val_loss: 0.0021 - 1s/epoch - 5ms/step
Epoch 35/200
230/230 - 1s - loss: 0.0014 - val_loss: 0.0021 - 1s/epoch - 5ms/step
Epoch 36/200
230/230 - 1s - loss: 0.0014 - val_loss: 0.0025 - 1s/epoch - 5ms/step
Epoch 37/200
230/230 - 1s - loss: 0.0014 - val_loss: 0.0018 - 1s/epoch - 5ms/step
Epoch 38/200
230/230 - 1s - loss: 0.0013 - val_loss: 0.0019 - 1s/epoch - 5ms/step
Epoch 39/200
230/230 - 1s - loss: 0.0013 - val_loss: 0.0019 - 1s/epoch - 5ms/step
Epoch 40/200
230/230 - 1s - loss: 0.0013 - val_loss: 0.0018 - 1s/epoch - 5ms/step
Epoch 41/200
230

Epoch 126/200
230/230 - 1s - loss: 6.9466e-04 - val_loss: 0.0011 - 1s/epoch - 5ms/step
Epoch 127/200
230/230 - 1s - loss: 6.7816e-04 - val_loss: 0.0011 - 1s/epoch - 5ms/step
Epoch 128/200
230/230 - 1s - loss: 6.7200e-04 - val_loss: 0.0010 - 1s/epoch - 5ms/step
Epoch 129/200
230/230 - 1s - loss: 6.7931e-04 - val_loss: 0.0011 - 1s/epoch - 6ms/step
Epoch 130/200
230/230 - 1s - loss: 6.7839e-04 - val_loss: 0.0011 - 1s/epoch - 5ms/step
Epoch 131/200
230/230 - 1s - loss: 6.6776e-04 - val_loss: 0.0011 - 1s/epoch - 5ms/step
Epoch 132/200
230/230 - 1s - loss: 6.7461e-04 - val_loss: 0.0011 - 1s/epoch - 5ms/step
Epoch 133/200
230/230 - 1s - loss: 6.6088e-04 - val_loss: 0.0011 - 1s/epoch - 5ms/step
Epoch 134/200
230/230 - 1s - loss: 6.6374e-04 - val_loss: 0.0011 - 1s/epoch - 5ms/step
Epoch 135/200
230/230 - 1s - loss: 6.6291e-04 - val_loss: 0.0011 - 1s/epoch - 5ms/step
Epoch 136/200
230/230 - 1s - loss: 6.6393e-04 - val_loss: 0.0011 - 1s/epoch - 5ms/step
Epoch 137/200
230/230 - 1s - loss: 6.6683e-

## CONVLSTM-ATTNAE     


In [ ]:
for i in range(1,4):
    df_hyp = pd.read_csv('/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/Conv_LstmAttnAE_HYP.csv')
    PID_RESULTSPATH_TOTAL_TRAIN = f'/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/Results/Iter{i}/train/'
    PID_RESULTSPATH_TOTAL_TEST = f'/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/Results/Iter{i}/test/'
    callback1 = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, verbose=1)
    lookback = 8
    source_dict_results_test =  {}
    source_dict_results_train = {}
    for file in total_list:
        inv = f'inv_{file}'
        xtrain, xtest, ytrain, ytest = get_train_test_data(inv, xtrain_dfs, xtest_dfs, ytrain_dfs, ytest_dfs)
        xtrain = xtrain.reshape(xtrain.shape[0], 1, xtrain.shape[1])
        xtest = xtest.reshape(xtest.shape[0], 1, xtest.shape[1])
#         ytrain, ytest = ytrain[lookback+1:], ytest[lookback+1:]
        model__batch_size,model__encoding_dim, model__epochs, model__layer1, model__layer2 = getHyperParams(file, df_hyp)                                               
        model  = Conv_LstmAttnAEModel(layer1=model__layer1, layer2=model__layer2, encoding_dim=model__encoding_dim)
        model.fit(xtrain, xtrain, epochs=model__epochs, batch_size=model__batch_size, verbose=2,validation_split=0.1, callbacks=callback1)
        test_df, train_df = other_evaluation(xtrain, xtest, ytest, ytrain, model)
        source_dict_results_test[file] = test_df
        source_dict_results_train[file] = train_df
    with open(PID_RESULTSPATH_TOTAL_TEST+'test_STL_Conv_LstmAttnAE.pkl', 'wb') as f:
        pickle.dump(source_dict_results_test, f)
    with open(PID_RESULTSPATH_TOTAL_TRAIN+'train_STL_Conv_LstmAttnAE.pkl', 'wb') as f:
        pickle.dump(source_dict_results_train, f)
    train_merge_df = data_unimodel(source_list, xtrain_dfs)
    random.shuffle(train_merge_df)
    train_merge_df = train_merge_df.reshape(train_merge_df.shape[0], 1, train_merge_df.shape[1])

    model = Conv_LstmAttnAEModel(layer1=64, layer2=32, encoding_dim=32)
    model.fit(train_merge_df, train_merge_df, epochs=200, batch_size=128, verbose=2,validation_split=0.1, callbacks=callback1)
    lookback = 8
    mtl_dict_results_test =  {}
    mtl_dict_results_train = {}
    for file in total_list:
        inv = 'inv_'+str(file)
        xtrain, xtest, ytrain, ytest = get_train_test_data(inv, xtrain_dfs, xtest_dfs, ytrain_dfs, ytest_dfs)
        xtrain = xtrain.reshape(xtrain.shape[0], 1, xtrain.shape[1])
        xtest = xtest.reshape(xtest.shape[0], 1, xtest.shape[1])
#         ytrain, ytest = ytrain[lookback+1:], ytest[lookback+1:]
        test_df, train_df = other_evaluation(xtrain, xtest, ytest, ytrain, model)
        mtl_dict_results_test[file] = test_df
        mtl_dict_results_train[file] = train_df
    with open(PID_RESULTSPATH_TOTAL_TEST+'test_MTL_Conv_LstmAttnAE.pkl', 'wb') as f:
        pickle.dump(mtl_dict_results_test, f)
    with open(PID_RESULTSPATH_TOTAL_TRAIN+'train_MTL_Conv_LstmAttnAE.pkl', 'wb') as f:
        pickle.dump(mtl_dict_results_train, f)